In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.contrib import rnn
import random
import collections
import time
from __future__ import print_function

%matplotlib inline

In [20]:
start_time = time.time()
def elapsed(sec):
    if sec<60:
        return str(sec) + " sec"
    elif sec<(60*60):
        return str(sec/60) + " min"
    else:
        return str(sec/(60*60)) + " hr"

In [3]:
# Target log path
logs_path = '/rnn_words'
writer = tf.summary.FileWriter(logs_path)

In [5]:
train = 'sample.txt'

In [10]:
def read_data(file_name):
    with open(file_name) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    content = [content[i].split() for i in range(len(content))]
    content = np.array(content)
    content = np.reshape(content, [-1, ])
    return content

In [11]:
train_data = read_data(train)

In [12]:
train_data

array(['long', 'ago', ',', 'the', 'mice', 'had', 'a', 'general', 'council',
       'to', 'consider', 'what', 'measures', 'they', 'could', 'take', 'to',
       'outwit', 'their', 'common', 'enemy', ',', 'the', 'cat', '.',
       'some', 'said', 'this', ',', 'and', 'some', 'said', 'that', 'but',
       'at', 'last', 'a', 'young', 'mouse', 'got', 'up', 'and', 'said',
       'he', 'had', 'a', 'proposal', 'to', 'make', ',', 'which', 'he',
       'thought', 'would', 'meet', 'the', 'case', '.', 'you', 'will',
       'all', 'agree', ',', 'said', 'he', ',', 'that', 'our', 'chief',
       'danger', 'consists', 'in', 'the', 'sly', 'and', 'treacherous',
       'manner', 'in', 'which', 'the', 'enemy', 'approaches', 'us', '.',
       'now', ',', 'if', 'we', 'could', 'receive', 'some', 'signal', 'of',
       'her', 'approach', ',', 'we', 'could', 'easily', 'escape', 'from',
       'her', '.', 'i', 'venture', ',', 'therefore', ',', 'to', 'propose',
       'that', 'a', 'small', 'bell', 'be', 'procured'

In [14]:
def build_dataset(words):
    count = collections.Counter(words).most_common()
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary

dictionary, reverse_dictionary = build_dataset(train_data)
vocab_size = len(dictionary)

In [15]:
# Parameters
learning_rate = 0.001
training_iters = 50000
display_step = 1000
n_input = 3

# number of units in RNN cell
n_hidden = 512

# tf Graph input
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocab_size])

# RNN output node weights and biases
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))
}
biases = {
    'out': tf.Variable(tf.random_normal([vocab_size]))
}


In [16]:
def RNN(x, weights, biases):

    # reshape to [1, n_input]
    x = tf.reshape(x, [-1, n_input])

    # Generate a n_input-element sequence of inputs
    # (eg. [had] [a] [general] -> [20] [6] [33])
    x = tf.split(x,n_input,1)

    # 2-layer LSTM, each layer has n_hidden units.
    # Average Accuracy= 95.20% at 50k iter
    rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden)])

    # 1-layer LSTM with n_hidden units but with lower accuracy.
    # Uncomment line below to test but comment out the 2-layer rnn.MultiRNNCell above
    # rnn_cell = rnn.BasicLSTMCell(n_hidden)

    # generate prediction
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)

    # there are n_input outputs but
    # we only want the last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']


In [17]:
pred = RNN(x, weights, biases)

In [18]:
# Loss function and Optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

In [19]:
# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

#Initialize variables
init = tf.global_variables_initializer()

In [23]:
# Running the graph

with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_input+1)
    end_offset = n_input + 1
    acc_total = 0
    loss_total = 0

    writer.add_graph(session.graph)

    while step < training_iters:
        if offset > (len(train_data) - end_offset):
             offset = random.randint(0, n_input+1)
        symbols_in_keys = [ [dictionary[ str(train_data[i])]] for i in range(offset, offset+n_input) ]
        symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])

        symbols_out_onehot = np.zeros([vocab_size], dtype=float)
        symbols_out_onehot[dictionary[str(train_data[offset+n_input])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("Iter= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [train_data[i] for i in range(offset, offset + n_input)]
            symbols_out = train_data[offset + n_input]
            symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
            print("%s - [%s] vs [%s]" % (symbols_in,symbols_out,symbols_out_pred))
        step += 1
        offset += (n_input+1)
    print("Optimization Finished!")
    print("Elapsed time: ", elapsed(time.time() - start_time))
    print("Run on command line.")
    print("\ttensorboard --logdir=%s" % (logs_path))
    print("Point your web browser to: http://localhost:6006/")
    
    while True:
        prompt = "%s words: " % n_input
        sentence = input(prompt)
        sentence = sentence.strip()
        words = sentence.split(' ')
        if len(words) != n_input:
            continue
        try:
            symbols_in_keys = [dictionary[str(words[i])] for i in range(len(words))]
            for i in range(32):
                keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])
                onehot_pred = session.run(pred, feed_dict={x: keys})
                onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
                sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
                symbols_in_keys = symbols_in_keys[1:]
                symbols_in_keys.append(onehot_pred_index)
            print(sentence)
        except:
            print("Word not in dictionary")

Iter= 1000, Average Loss= 4.443420, Average Accuracy= 3.10%
['the', 'old', 'mouse'] - [said] vs [means]
Iter= 2000, Average Loss= 2.751008, Average Accuracy= 24.50%
['and', 'nobody', 'spoke'] - [.] vs [this]
Iter= 3000, Average Loss= 2.476694, Average Accuracy= 32.00%
['is', 'all', 'very'] - [well] vs [with]
Iter= 4000, Average Loss= 2.060927, Average Accuracy= 47.50%
['old', 'mouse', 'got'] - [up] vs [up]
Iter= 5000, Average Loss= 1.798270, Average Accuracy= 50.60%
['this', 'proposal', 'met'] - [with] vs [from]
Iter= 6000, Average Loss= 1.567409, Average Accuracy= 55.40%
['a', 'ribbon', 'round'] - [the] vs [the]
Iter= 7000, Average Loss= 1.398148, Average Accuracy= 61.20%
['receive', 'some', 'signal'] - [of] vs [of]
Iter= 8000, Average Loss= 1.432168, Average Accuracy= 61.30%
['treacherous', 'manner', 'in'] - [which] vs [the]
Iter= 9000, Average Loss= 1.304113, Average Accuracy= 64.00%
['agree', ',', 'said'] - [he] vs [he]
Iter= 10000, Average Loss= 1.078658, Average Accuracy= 72.70%


KeyboardInterrupt: 